In [1]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# internal packages src/
from utils.load_and_preprocess import load_and_preprocess, missing_data, create_features, split_train_test
from models.sarima import train_model_sarima, predict_sarima
from utils.plot import plot_method1, plot_method2, plot_method3

In [3]:
merged_df = load_and_preprocess('../data/sell_data_cleaned.csv', '../data/marketing_campaign.csv', '../data/stock.csv')
missing_data(merged_df)

,Total,Percent,Types
ID,0,0.0,int64
Product,0,0.0,object
Date,0,0.0,datetime64[ns]
Quantity,0,0.0,float64
Country,0,0.0,object
CountryStatus,0,0.0,object
ProductStatus,0,0.0,object
Stock,0,0.0,float64
ActiveCampaign,0,0.0,float64


In [4]:
featured_df = create_features(merged_df)
missing_data(featured_df)

,Total,Percent,Types
ID,0,0.0,int64
Product,0,0.0,object
Date,0,0.0,datetime64[ns]
Quantity,0,0.0,float64
Country,0,0.0,object
CountryStatus,0,0.0,int64
ProductStatus,0,0.0,int64
Stock,0,0.0,float64
ActiveCampaign,0,0.0,float64
Year,0,0.0,int32


In [5]:
train_df, test_df, cutoff_date = split_train_test(featured_df)

# First ensure proper datetime frequency
train_df_cz = train_df[train_df['Country'] == "CZ        "].copy()

# Convert to datetime and set frequency properly
train_df_cz['Date'] = pd.to_datetime(train_df_cz['Date'])
train_df_cz = train_df_cz.set_index('Date')  # Weekly frequency

# Alternative if you need start-of-week dates
train_df_cz.index = train_df_cz.index.to_period('W').to_timestamp()

# Verify frequency
print(train_df_cz.index.freq)  # Should show 'W' or similar

# Handle any missing periods if needed
train_df_cz = train_df_cz.fillna(method='ffill')


None


C:\Users\rafae\AppData\Local\Temp\ipykernel_4508\305548506.py:17: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  train_df_cz = train_df_cz.fillna(method='ffill')


In [ ]:
train_df, test_df, cutoff_date = split_train_test(featured_df)
# train_df.head()

# missing_data(train_df)
# too heavy
# features = [
#     'Year', 'Month', 'Week', 'DayOfWeek', 'DayOfMonth',
#     'CountryEncoded', 'ProductEncoded', 'CountryStatus', 'ProductStatus',
#     'ActiveCampaign', 'Stock', 'StockLag1', 'StockLag7', 'StockMA7', 'StockMA14'
# ]

features = [
    'Month', 'DayOfWeek', 
    'CountryStatus', 'ActiveCampaign', 
    'Stock', 'StockLag1', 'StockMA7'
]

# First ensure proper datetime frequency
train_df_cz = train_df[train_df['Country'] == "CZ        "].copy()

# Convert to datetime and set frequency properly
train_df_cz['Date'] = pd.to_datetime(train_df_cz['Date'])
train_df_cz = train_df_cz.set_index('Date')  # Weekly frequency

# Alternative if you need start-of-week dates
train_df_cz.index = pd.DatetimeIndex(train_df_cz.index).to_period('W')

# Verify frequency
print(train_df_cz.index.freq)  # Should show 'W' or similar

# Handle any missing periods if needed
train_df_cz = train_df_cz.fillna(method='ffill')

# print(train_df.columns.tolist())
model = train_model_sarima(train_df_cz, features)

<Week: weekday=6>


C:\Users\rafae\AppData\Local\Temp\ipykernel_2012\1641275211.py:32: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  train_df_cz = train_df_cz.fillna(method='ffill')
c:\Users\rafae\Desktop\ujep\malfini\HackathonHuita\.venv\Lib\site-packages\statsmodels\tsa\statespace\representation.py:374: FutureWarning: Unknown keyword arguments: dict_keys(['low_memory']).Passing unknown keyword arguments will raise a TypeError beginning in version 0.15.
  warnings.warn(msg, FutureWarning)


MemoryError: Unable to allocate 31.4 GiB for an array with shape (55, 55, 1392101) and data type float64

: 

In [ ]:
test_df = predict_sarima(model, test_df, features)

rmse = np.sqrt(mean_squared_error(test_df['Quantity'], test_df['Predicted']))
mae = mean_absolute_error(test_df['Quantity'], test_df['Predicted'])
print(f'RMSE: {rmse:.2f}, MAE: {mae:.2f}')

In [ ]:
plot_method1(train_df, test_df, cutoff_date)